In [4]:
import torch

In [5]:
from rgb_baseline import MyResnet
import rgb_baseline

In [6]:
net = MyResnet()
net = torch.load('rgb_resnet.pth')

C:\Users\morit\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import sys
sys.path.insert(1, 'C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/data_pipeline') # TODO
import os

path_ege_data = os.path.join("..",".." ,"data", "Dataset Ege")

print(path_ege_data)

# DATA ENGINEERING
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from data_sampler import WeightedSampler
from dataset import CARLADataset#, CARLADatasetMultiProcessing

config = {"used_inputs": ["rgb","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake"],
        "seq_len": 1
        }

batch_size = 16

preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


def normalize_batch(tensors):
    liste = []
    for tensor in tensors:
        tensor = preprocess(tensor ) #* 1/255
        liste.append(tensor)
    return torch.stack(liste)   

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_dataset = CARLADataset(root_dir=path_ege_data, config=config, transform=None)
weighted_sampler = WeightedSampler(dataset=train_dataset)

## TEST
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(next(iter(test_dataloader)).keys())
idx, X = next(enumerate(test_dataloader))
data_ = normalize_batch(X["rgb"].transpose(1,4)[:,:,:,:,0]).float().to(device)
target_ = (X["throttle"],X["steer"],X["brake"])
with torch.no_grad():
    outputs_ = net(data_)
print(outputs_[0])

In [ ]:
import sys
sys.path.insert(1, 'C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/data_pipeline') # TODO
import os

path_ege_data = os.path.join("..",".." ,"data", "Dataset Ege")

print(path_ege_data)

# DATA ENGINEERING
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from data_sampler import WeightedSampler
from dataset import CARLADataset#, CARLADatasetMultiProcessing

config = {"used_inputs": ["rgb","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake"],
        "seq_len": 1
        }

batch_size = 1

preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


def normalize_batch(tensors):
    liste = []
    for tensor in tensors:
        tensor = preprocess(tensor ) #* 1/255
        liste.append(tensor)
    return torch.stack(liste)   

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_dataset = CARLADataset(root_dir=path_ege_data, config=config, transform=None)
weighted_sampler = WeightedSampler(dataset=train_dataset)

## TEST
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(next(iter(test_dataloader)).keys())
idx, X = next(enumerate(test_dataloader))
data_ = normalize_batch(X["rgb"].transpose(1,4)[:,:,:,:,0]).float().to(device)
target_ = (X["throttle"],X["steer"],X["brake"])
with torch.no_grad():
    outputs_ = net(data_)
print(outputs_[0])

In [ ]:
# single rgb -> batch_rgb
single = data_[0,:,:,:]
batch = torch.unsqueeze(single, dim=0)

print(batch.shape)


In [ ]:
type(X["rgb"])